In [12]:
import os
import numpy as np
import pandas as pd
from PIL import Image, ImageStat, ImageEnhance

In [8]:
# read image paths into lists
# can iterate over lists to open images later

def read_img_dataset_paths(data_dir = 'images/asl_alphabet_train/'):
    img_paths = []

    for (dirpath, dirnames, filenames) in os.walk(data_dir):
        dirnames.sort()
        img_paths.extend([os.path.join(dirpath, f) for f in sorted(filenames)])

    return img_paths


In [9]:
train = read_img_dataset_paths('images/original/asl_alphabet_train/')
test = read_img_dataset_paths('images/original/asl_alphabet_test/')
print(f'Train dataset len: {len(train)} = \n\t{train[:5]}...')
print(f'Test dataset len: {len(test)} = \n\t{test[:5]}...')

Train dataset len: 87001 = 
	['images/original/asl_alphabet_train/.DS_Store', 'images/original/asl_alphabet_train/A/A1.jpg', 'images/original/asl_alphabet_train/A/A10.jpg', 'images/original/asl_alphabet_train/A/A100.jpg', 'images/original/asl_alphabet_train/A/A1000.jpg']...
Test dataset len: 28 = 
	['images/original/asl_alphabet_test/A_test.jpg', 'images/original/asl_alphabet_test/B_test.jpg', 'images/original/asl_alphabet_test/C_test.jpg', 'images/original/asl_alphabet_test/D_test.jpg', 'images/original/asl_alphabet_test/E_test.jpg']...


In [13]:
def downsample_imgs(train_img_paths):
    for i, path in enumerate(train_img_paths):
        if 'DS_Store' in path:
            continue
        image = Image.open(path)
        if i % 10 == 0:
            og_path = path.split('/')
            new_dir = f"images/sampled/asl_alphabet_train/{og_path[-2]}"
            os.makedirs(new_dir, exist_ok=True)
            enhancer = ImageEnhance.Sharpness(image)
            image = enhancer.enhance(4) 

            # Improve contrast
            enhancer = ImageEnhance.Contrast(image)
            image = enhancer.enhance(2)

            image = image.convert('L')

            image.save(f"{new_dir}/{og_path[-1]}")
            image.close()

downsample_imgs(train)

In [7]:
from PIL import Image, ImageStat

# update with more EDA stuff...
def train_img_stats(train_img_paths):
    headers = ['path', 'class', 'width', 'height', 'avg_brightness']
    data = []

    for i in train_img_paths:
        image = Image.open(i)

        # get average pixel brightness + possibly other image stats by converting to greyscale first
        stats = ImageStat.Stat(image.convert('L'))
        avg_brightness = stats.mean[0]
        data.append([i, i.split('/')[-2], image.width, image.height, avg_brightness]) # classes = 'A', 'B', 'C'...
        image.close()

    df = pd.DataFrame(data, columns=headers)

    return df

In [8]:
train_df = train_img_stats(train)

In [9]:
train_df

,path,class,width,height,avg_brightness
0,images/asl_alphabet_train/A/A1.jpg,A,200,200,113.838675
1,images/asl_alphabet_train/A/A10.jpg,A,200,200,113.838125
2,images/asl_alphabet_train/A/A100.jpg,A,200,200,105.411575
3,images/asl_alphabet_train/A/A1000.jpg,A,200,200,141.798375
4,images/asl_alphabet_train/A/A1001.jpg,A,200,200,141.975950
...,...,...,...,...,...
86995,images/asl_alphabet_train/space/space995.jpg,space,200,200,135.174750
86996,images/asl_alphabet_train/space/space996.jpg,space,200,200,135.613300
86997,images/asl_alphabet_train/space/space997.jpg,space,200,200,136.018650
86998,images/asl_alphabet_train/space/space998.jpg,space,200,200,136.241350
